# Analyzing Neural Time Series Data

#### Data extraction from MATLAB matrix as well as some functions written by Andrew J. Graves on 12/03/19

### Get the data

In [1]:
# Import modules
from scipy import io
import numpy as np
import mne
# My version of MNE is 0.19.2- AJG
print(mne.__version__)

# Load Sample MATLAB/EEGLAB object from current directory
EEG = io.loadmat('sampleEEGdata.mat')['EEG']

# Get the data and reshape data from 3D (chan x time x trial) to 2D (chan x (time x trial))
eeg_data = EEG['data'].item()
two_d = eeg_data.reshape(eeg_data.shape[0], (eeg_data.shape[1]*eeg_data.shape[2]))

# Extract sampling rate, channel names, channel types, and time
samp_rate = EEG['srate'].item()
chan_names = sum(np.concatenate(EEG['chanlocs'].item()['labels'].tolist()).tolist(), [])
chan_types = ['eeg'] * len(chan_names)
eeg_time = EEG['times'].item()[0]

# Create an MNE info object and combine 2D data with info for an MNE raw object
mont = mne.channels.make_standard_montage('biosemi64')
info = mne.create_info(ch_names=chan_names, sfreq=samp_rate, ch_types=chan_types, montage=mont)
raw = mne.io.RawArray(two_d, info, verbose=False)

# Extract various features useful for plots
epochs = EEG['epoch'].item()[0]
chan_theta = EEG['chanlocs'].item()['theta']
chan_radius = EEG['chanlocs'].item()['radius'].tolist()

0.19.2


### Get useful functions

In [2]:
# ERP
def get_erp(three_d, chan_index):
    return np.squeeze(np.mean(three_d[chan_index, :, :], 2))

# Low-pass filter for ERPs
def low_pass_erp(erp, filter_cutoff, trans_width, nyq):
    f_freqs = np.array([0, filter_cutoff, filter_cutoff * (1 + trans_width), nyq]) / nyq
    ideal_resp = np.array([1, 1, 0, 0])
    filter_weights = signal.firls(101, f_freqs, ideal_resp)
    return signal.filtfilt(filter_weights, 1, erp)

# Band-pass filter for ERPs
def band_pass_erp(erp, filter_low, filter_high, trans_width, nyq):
    f_freqs = np.array([0, filter_low * (1 - trans_width), filter_low, 
                        filter_high, filter_high * (1 + trans_width), nyq]) / nyq
    ideal_resp = np.array([0, 0, 1, 1, 0, 0])
    filter_weights = signal.firls(round(3 * (samp_rate.item() / filter_low) + 1), f_freqs, ideal_resp)
    return signal.filtfilt(filter_weights, 1, erp)

# Convert from Polar to Cartesian coordinates
def pol_to_cart(phi, rho):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

# Generate a sine wave
def get_sine_wave(frequency, amplitude, time_vec, phase=0):
    return amplitude * np.sin(2 * np.pi * frequency * time_vec + phase)

# Compute power with numpy fourier transform
# Note that Cohen multiplied the series by 2 in the code rather than squaring it for some reason.
# AFAIK squaring is appropriate, which is why the magnitudes of my frequencies are larger
def compute_power(signal, srate):
    frequencies = np.linspace(0, int(srate / 2), int(len(signal) / 2) + 1)
    fft_sig = np.fft.fft(signal) / len(signal)
    return np.abs(fft_sig[0:len(frequencies)]) ** 2, frequencies

# Compute phase with numpy fourier transform
def compute_phase(signal, srate):
    frequencies = np.linspace(0, int(srate / 2), int(len(signal) / 2) + 1)
    fft_sig = np.fft.fft(signal) / len(signal)
    return np.angle(fft_sig[0:len(frequencies)], deg=False), frequencies